<a href="https://colab.research.google.com/github/morshedik/Cancer-Classification-Project/blob/main/Step4_ModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/TCGA_Data"

# Model

In [ ]:
import pandas as pd

# Load preprocessed data
data_path = '/content/drive/My Drive/Colab Notebooks/TCGA_Data/preprocessed_data.csv'
preprocessed_data = pd.read_csv(data_path)

# Separate features and labels
X = preprocessed_data.drop(columns=['sample', 'primary_disease'])
y = preprocessed_data['primary_disease']

# Split into train and test (you already did this, but let’s confirm)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes
print("Training Set Shape:", X_train.shape)  # Should be (1848, 100)
print("Testing Set Shape:", X_test.shape)    # Should be (463, 100)
print("Training Labels Sample:", y_train.head())

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
# Create the model (multi_class='ovr' for one-vs-rest)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
# Train on the training data
model.fit(X_train, y_train)

# Print a confirmation
print("Model trained successfully!")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Print some predictions
print("Sample Predictions:")
for i in range(5):
    print(f"Sample {i}: Predicted {y_pred[i]}, Actual {y_test.iloc[i]}")


from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {accuracy:.2f}")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Cancer Classification")
plt.show()

# Modified model


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load data(top 100 high variance genes)
data_path = '/content/drive/My Drive/Colab Notebooks/TCGA_Data/preprocessed_data.csv'
preprocessed_data = pd.read_csv(data_path)
# Separate features and labels
X = preprocessed_data.drop(columns=['sample', 'primary_disease'])
y = preprocessed_data['primary_disease']

# Split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train accuracy (check for overfitting)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Model Accuracy on Training Set: {train_accuracy:.2f}")

# Test accuracy
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {test_accuracy:.2f}")

# Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Average CV Accuracy: {cv_scores.mean():.2f} (+/- {cv_scores.std() * 2:.2f})")

# Feature scaling and re-train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model_scaled = LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42)
model_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = model_scaled.predict(X_test_scaled)
accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
print(f"Model Accuracy on Scaled Test Set: {accuracy_scaled:.2f}")

# Regularization (try C=0.1)
model_regularized = LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42, C=0.1)
model_regularized.fit(X_train, y_train)
y_pred_regularized = model_regularized.predict(X_test)
accuracy_regularized = accuracy_score(y_test, y_pred_regularized)
print(f"Model Accuracy with Regularization (C=0.1): {accuracy_regularized:.2f}")

# Check for data leakage
train_samples = set(X_train.index)
test_samples = set(X_test.index)
overlap = train_samples.intersection(test_samples)
print(f"Number of Overlapping Samples: {len(overlap)}")
if len(overlap) > 0:
    print("Overlapping Samples:", overlap)

# Confusion matrix (optional, for insight)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Cancer Classification")
plt.show()

# Save the model
import joblib

joblib.dump(model, '/content/drive/My Drive/Colab Notebooks/TCGA_Data/cancer_classifier_step4.pkl')
print("Model saved to Google Drive.")

# Perfect Accuracy Explanation
The model achieves 100% accuracy due to strong biological signals, such as KLK3 (high in prostate adenocarcinoma) and SFTPB (high in lung adenocarcinoma), which create clear separation among BRCA, LUAD, and PRAD. This is supported by visualizations (e.g., expression scatter plots) and mean expression differences (KLK3: 18.23 PRAD vs. 1.39 BRCA, 0.36 LUAD). However, to rule out overfitting or artifacts (e.g., batch effects), we will validate on external data in Step 5.

# Overfitting Check

In [ ]:
print("Mean Expression by Cancer Type for KLK3:")
print(preprocessed_data.groupby('primary_disease')['KLK3'].mean())

print("Test Set Class Distribution:")
print(y_test.value_counts())

import numpy as np
correlation_matrix = X_train.corr()
print("Correlation of Top 5 Features with KLK3:")
print(correlation_matrix['KLK3'].nlargest(5))

# prompt: expression of TMEFF2 and KLK3 in different cancer types

print("Mean Expression by Cancer Type for TMEFF2:")
print(preprocessed_data.groupby('primary_disease')['TMEFF2'].mean())

print("Mean Expression by Cancer Type for KLK3:")
print(preprocessed_data.groupby('primary_disease')['KLK3'].mean())

# Reduce to top 50 genes (less complex model)
top_50_genes = X.var().nlargest(50).index
X_50 = X[top_50_genes]
X_train_50, X_test_50, y_train, y_test = train_test_split(X_50, y, test_size=0.2, random_state=42, stratify=y)

# Train and evaluate
model_50 = LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42)
model_50.fit(X_train_50, y_train)
y_pred_50 = model_50.predict(X_test_50)
accuracy_50 = accuracy_score(y_test, y_pred_50)
print(f"Model Accuracy with 50 Genes: {accuracy_50:.2f}")

from sklearn.multiclass import OneVsRestClassifier

# Use OneVsRestClassifier (recommended for scikit-learn 1.5+)
base_lr = LogisticRegression(max_iter=1000, random_state=42)
model_ovr = OneVsRestClassifier(base_lr)
model_ovr.fit(X_train, y_train)
y_pred_ovr = model_ovr.predict(X_test)
accuracy_ovr = accuracy_score(y_test, y_pred_ovr)
print(f"Model Accuracy with OneVsRestClassifier: {accuracy_ovr:.2f}")

import numpy as np

# Correlation matrix
correlation_matrix = X_train.corr()

# Set threshold for correlation (e.g., >0.9)
threshold = 0.9
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Drop highly correlated features
X_train_reduced = X_train.drop(to_drop, axis=1)
X_test_reduced = X_test.drop(to_drop, axis=1)

# Get coefficients from the OneVsRestClassifier (first classifier for BRCA vs. others)
coef = model.estimators_[0].coef_[0]  # First class (BRCA) coefficients
feature_importance = pd.DataFrame({'Gene': X_train.columns, 'Coefficient': coef})
print("Top 10 Most Important Features for BRCA:")
print(feature_importance.sort_values(by='Coefficient', key=abs, ascending=False).head(10))

# Train and evaluate
model_reduced = LogisticRegression(max_iter=1000, random_state=42)
model_reduced.fit(X_train_reduced, y_train)
y_pred_reduced = model_reduced.predict(X_test_reduced)
accuracy_reduced = accuracy_score(y_test, y_pred_reduced)
print(f"Model Accuracy with Reduced Features: {accuracy_reduced:.2f}")
print("Dropped Features:", to_drop)

from sklearn.preprocessing import StandardScaler

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train with stronger regularization
model_scaled_reg = LogisticRegression(max_iter=1000, C=0.01, random_state=42)
model_scaled_reg.fit(X_train_scaled, y_train)
y_pred_scaled_reg = model_scaled_reg.predict(X_test_scaled)
accuracy_scaled_reg = accuracy_score(y_test, y_pred_scaled_reg)
print(f"Model Accuracy with Scaling and C=0.01: {accuracy_scaled_reg:.2f}")

from sklearn.decomposition import PCA

# Reduce to 30 genes (simpler)
top_30_genes = X.var().nlargest(30).index
X_30 = X[top_30_genes]
X_train_30, X_test_30, _, _ = train_test_split(X_30, y, test_size=0.2, random_state=42, stratify=y)

model_30 = LogisticRegression(max_iter=1000, random_state=42)
model_30.fit(X_train_30, y_train)
y_pred_30 = model_30.predict(X_test_30)
accuracy_30 = accuracy_score(y_test, y_pred_30)
print(f"Model Accuracy with 30 Genes: {accuracy_30:.2f}")

# Or use PCA (50 components)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, _, _ = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)

model_pca = LogisticRegression(max_iter=1000, random_state=42)
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f"Model Accuracy with PCA (50 components): {accuracy_pca:.2f}")

# prompt: Look at sample IDs in your train and test sets. If any share the same first 12 characters (e.g., TCGA-2A-A8VL)

# Check for overlapping sample IDs in train and test sets
train_samples = set(X_train.index.astype(str).str[:12])
test_samples = set(X_test.index.astype(str).str[:12])

overlap = train_samples.intersection(test_samples)

if overlap:
    print("Overlapping sample IDs (first 12 characters):")
    for sample_id in overlap:
        print(sample_id)
else:
    print("No overlapping sample IDs found in train and test sets.")

# prompt: Check metadata for batch IDs or sequencing platforms. If cancer types align with batches, the model might be learning artifacts.
# Assuming 'preprocessed_data' DataFrame is already loaded
# Check for metadata columns related to batches or sequencing platforms
metadata_columns = ['batch_id', 'sequencing_platform', 'experiment_id']  # Add other relevant columns
metadata_columns = [col for col in metadata_columns if col in preprocessed_data.columns]

if metadata_columns:
    print("Metadata columns found:")
    for col in metadata_columns:
        print(col)

        # Analyze relationship between metadata and cancer types
        print("\nDistribution of cancer types within each batch:")
        print(preprocessed_data.groupby(col)['primary_disease'].value_counts())

        # Further analysis (e.g., chi-squared test) can be performed to check for statistically significant associations
else:
    print("No metadata columns related to batches or sequencing platform were found.")

    # prompt: Plot expression levels of top genes (e.g., KLK3, SFTPB) by cancer type. If there’s no overlap (e.g., PRAD’s KLK3 is always >10 while BRCA/LUAD is <2), perfect separation is likely.

import matplotlib.pyplot as plt

# Assuming 'preprocessed_data' DataFrame is already loaded

# Select the genes of interest
genes = ['KLK3', 'SFTPB']  # Add other genes

# Create the plot
plt.figure(figsize=(10, 6))

for gene in genes:
    if gene in preprocessed_data.columns:  # Check if the gene exists in the dataframe
        for cancer_type in preprocessed_data['primary_disease'].unique():
            expression_levels = preprocessed_data[preprocessed_data['primary_disease'] == cancer_type][gene]
            plt.scatter(
                [cancer_type] * len(expression_levels),
                expression_levels,
                label=f"{gene} ({cancer_type})"
            )

plt.xlabel("Cancer Type")
plt.ylabel("Expression Level")
plt.title("Expression Levels of Top Genes by Cancer Type")
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()


#separability with boxplots
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(x='primary_disease', y='KLK3', data=preprocessed_data)
plt.title('KLK3 Expression by Cancer Type')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='primary_disease', y='SFTPB', data=preprocessed_data)
plt.title('SFTPB Expression by Cancer Type')
plt.xticks(rotation=45)
plt.show()

import seaborn as sns
plt.figure(figsize=(12, 8))
sns.heatmap(X_train.corr(), cmap='coolwarm', center=0, annot=False)
plt.title('Correlation Heatmap of Top 100 Genes')
plt.show()
